In [ ]:
import xmltodict # to convert the raw metadata from xml format to dict
import pandas as pd # final format of stored data
from sickle import Sickle # to retrieve data from the OAI arxiv interface
import xmltodict
import json
import urllib
import csv

# Connect to arXiv’s OAI interface
connection = Sickle('http://export.arxiv.org/oai2')
print('Getting papers...')
data = connection.ListRecords(**{'metadataPrefix': 'arXiv', 'from': '2019-01-01', 'until': '2019-01-01','set':'cs', 'ignore_deleted': True})
print('Papers retrieved.')

iters = 0
# Store raw metadata in .txt file
with open('arXiv_metadata_raw.txt','a+') as f:
    while True:
        try:
            f.write(data.next().raw)
            errors = 0
            iters +=1
            
            if iters % 10000 == 0:
                print('On iter', iters)
        
        except AttributeError:
            if errors >5:
                raise AttributeError('\nQUITTING: Too many sequential errors\n')
            else:
                print('\nERROR!\n')
                errors +=1
                
        except StopIteration:
            print('On iter', iters)
            print('\nDONE!')
            break
# Format raw metadata
raw_data = ''

with open('arXiv_metadata_raw.txt','r') as f:
    while True:
        data = f.read(100_000_000)
        if not data:
            break
        else:
            raw_data += data
            


def convert_dict(record_xml):
    record_dict = xmltodict.parse(record_xml, process_namespaces=False)['record']['metadata']['arXiv']
    
    record_dict['id'] = str(record_dict['id'])
    
    if not isinstance(record_dict['authors']['author'], list):
        authors = [record_dict['authors']['author']]
    else:
        authors = record_dict['authors']['author']
    
    authors = [(author['forenames'] + ' ' if 'forenames' in author.keys() else '') + author['keyname'] for author in authors]
        
    record_dict['authors'] = authors
    return record_dict

list_of_xml = raw_data.split('</record>')
list_of_xml = [_ + '</record>' for _ in list_of_xml]
list_of_dicts =[convert_dict(list_of_xml[i]) for i in range(1,len(list_of_xml)-1)]

# Export data as a pandas DataFrame
df = pd.DataFrame(list_of_dicts)
df.head()
df = df.drop(columns = ['@xmlns', '@xmlns:xsi', '@xsi:schemaLocation'])
df.to_csv('arXiv_metadata_formatted.csv')


df = pd.DataFrame(list_of_dicts, columns= ['title'])

s= df.size  #pdf count

result = {'folder-name': 'PDF Folder',"pdf-count":str(s),"documents":df}
jstr = json.dumps(result,
                   default=lambda df: json.loads(df.to_json()))

with open('Output folder/Metadata.json','w') as f:
    f.write(jstr)
    
    
# download pdf
with open('arXiv_metadata_formatted.csv','r') as csvfile:
    reader = csv.reader(csvfile)
    header = next(reader)
    i=0
    for row in reader:
        x = row[1]
        print(x)
        url = r"https://arxiv.org/pdf/"+x+".pdf"
        path = r"Output folder/PDF folder/"+str(i)+".pdf"
        i=i+1
        data = urllib.request.urlopen(url).read()
        with open(path,'wb') as f:
            f.write(data)
            f.close()
            print ("finish download " + url + "!")


